This is a demonstration of the features of IJuliaTimeMachine.

# What IJulia Provides

IJulia already provides some historical information. `In` records the inputs to cells, and `Out` records their outputs.  `IJulia.n` is the number of the current cell.

In [1]:
1+1

2

In [2]:
In[1]

"1+1"

In [3]:
Out[1]

2

# Setting up IJuliaTimeMachine

I recommend assigning `IJuliaTimeMachine` a shorter name, like `TM` as below.

In [4]:
using IJuliaTimeMachine
TM = IJuliaTimeMachine

IJuliaTimeMachine

# Recalling past states

In [5]:
x = 1+1

2

In [6]:
x = randn(3)

3-element Array{Float64,1}:
  0.7883376083383903
 -0.3227728304284359
  1.1642447725348362

Say that I just accidentally change the value of the variable `x`, and I'd like to know what it's value was after cell 5.  I can recover the state of variables from then with `@past`.

In [7]:
TM.@past 5
x

2

The value of `ans` was also recalled.

In [8]:
ans

2

The Time Machine stores a `deepcopy` of every variable.  While this is inefficient, it allows us to recover a vector, even if some other cell changes one of its entries.

In [9]:
vec = collect(1:4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [10]:
vec[3] = 100
vec

4-element Array{Int64,1}:
   1
   2
 100
   4

In [11]:
TM.@past 9

4-element Array{Int64,1}:
 1
 2
 3
 4

By default `@past` also recalls the output of the past cell, and so that output appears in the display.

In [12]:
vec

4-element Array{Int64,1}:
 1
 2
 3
 4

This can be very useful because IJulia's Out stores a pointer to an array, rather than the array.
This means that the value recalled can be changed, like this.

In [13]:
y = [1;2;3]

3-element Array{Int64,1}:
 1
 2
 3

In [14]:
Out[13]

3-element Array{Int64,1}:
 1
 2
 3

In [15]:
y[3] = 0
Out[13]

3-element Array{Int64,1}:
 1
 2
 0

Note that the last element changed to a 0. This does not happen with the values stored by the Time Machine.

In [16]:
TM.@past 13

3-element Array{Int64,1}:
 1
 2
 3

The Time Machine only stores variables it can effectively copy.
Right now, it can not copy functions.  

In [17]:
cell = IJulia.n
f(x) = x
f(1)

1

In [18]:
y = 2
f(x) = 2x
f(1)

2

In [19]:
TM.@past cell

1

In [20]:
f(1)

2

But, if the only thing that changes in a function is a global variable, then you can essentially recover the function.

In [21]:
f(x) = y*x

f (generic function with 1 method)

In [22]:
cell = IJulia.n
y = 3
f(1)

3

In [23]:
y = 4
f(1)

4

In [24]:
TM.@past cell

3

In [25]:
f(1)

3

You can stop the Time Machine from saving.

In [26]:
TM.stop_saving()

save_state (generic function with 1 method)

In [27]:
z = "hello!"

"hello!"

In [28]:
TM.@past 27

LoadError: State 27 was not saved.

And, you can make it start saving again.

In [29]:
TM.start_saving()

1-element Array{Function,1}:
 save_state (generic function with 1 method)

In [30]:
z = "hi :)"

"hi :)"

In [31]:
z = "overwrite that"

"overwrite that"

In [32]:
TM.@past 30
z

"hi :)"

If we really want to forget the past, or just save memory, we can clear some history.  Just give a list of the cells to be cleared.  If no list is specified, it clears all of them.

In [33]:
TM.clear_past([30])

In [34]:
TM.@past 30

LoadError: State 30 was not saved.

If you want to see exactly what was stored, or exactly what variables are being overwritten, look at `TM.past`.

In [35]:
TM.past[23].vars

Dict{Any,Any} with 4 entries:
  :x    => 2
  :y    => 4
  :cell => 22
  :vec  => [1, 2, 3, 4]

In [36]:
TM.past[23].ans

4

# Running big jobs in threads

The other feature of Time Machine is that it lets you run intensive jobs in threads, so that you can get other work done while they are running.  If you have a multicore machine, you can also view this as a way to manage running a bunch of experiments from Jupyter.  The key is to wrap the jobs in `TM.@thread begin`, followed by `end`.  Jobs that are running inside a `@thread` block work on sandboxed variables. They start by copying all variables the exist when they are called. But, they do not change the values of any variables. To access the values of the variables they change, use `@past`.

When the jobs finish, their result is stored in `Out`, and you can access their state from `past`.
Unfortunately, if the jobs contain any print statements, they can show up in other cells.

To see which jobs are running, look at `TM.running`.  `TM.finished` contains a list of those that have finished.

In the examples below, I will simulate the delay of a long-running job with `sleep`.  As you will see, results will change after jobs finish.

In [37]:
x = collect(1:3)
y = 1

1

In [38]:
t0 = time()
n = IJulia.n
TM.@thread begin
    y = y + 1
    push!(x,y)
    sleep(5)
    sum(y)
end

Task (runnable) @0x000000011f426ad0

In [39]:
println("After $(time() - t0) seconds.")
Out[n]

After 0.7151029109954834 seconds.


Task (runnable) @0x000000011f426ad0

In [40]:
sleep(10)

In [41]:
println("After $(time() - t0) seconds.")
Out[n]

After 10.80537486076355 seconds.


2

In [42]:
x, y

([1, 2, 3], 1)

In [43]:
TM.@past n
x, y

([1, 2, 3, 2], 2)

There is a subtle reason that I put the `sleep(10)` statement on a separate line.
The output of finished jobs is only inserted into `Out` at the start of the execution of the first cell that is run after the job finishes. So, it is possible to go one cell without the output being correct.
If you want to test it, put the `sleep(10)`  inside the next cell, and then run the cells in quick succession.

You can not put two `@thread` statements into one cell.

In [44]:
n = IJulia.n
TM.@thread begin
    x = x + 1
    y[1] = 3
    sum(y)
end
TM.@thread begin
    x = x + 1
    y[1] = 4
    sum(y)
end

@thread can be called at most once per cell.


But, you can have many running at once.  That's the point!

In [45]:
function intense(n)
    sleep(n)
    println("I slept for $(n) seconds!")
    n^2
end

intense (generic function with 1 method)

In [46]:
t0 = time()
n1 = IJulia.n
TM.@thread intense(10)

Task (runnable) @0x000000011de95f90

In [47]:
println("After $(time()-t0) seconds")
n2 = IJulia.n
TM.@thread intense(11)

After 0.03081798553466797 seconds


Task (runnable) @0x000000011de961d0

In [48]:
println("After $(time()-t0) seconds")
n3 = IJulia.n
TM.@thread intense(2)

After 0.1841750144958496 seconds


Task (runnable) @0x000000011de96410

In [49]:
println("After $(time()-t0) seconds")
TM.running

After 0.2656099796295166 seconds


Set{Any} with 4 elements:
  47
  48
  44
  46

In [50]:
sleep(3)

I slept for 2 seconds!


In [51]:
println("After $(time()-t0) seconds")
TM.running

After 4.430108070373535 seconds


Set{Any} with 3 elements:
  47
  44
  46

In [52]:
println("After $(time()-t0) seconds")
TM.finished

After 4.483709096908569 seconds


2-element Array{Int64,1}:
 38
 48

In [53]:
sleep(10)
println("After $(time()-t0) seconds")
TM.running

I slept for 10 seconds!
I slept for 11 seconds!
After 14.552716970443726 seconds


Set{Any} with 1 element:
  44

In [54]:
Out[n1], Out[n2], Out[n3]

(100, 121, 4)

I don't know why cell 44 is listed as running.  That must be a bug!

In [57]:
TM.running

Set{Any} with 1 element:
  44

In [56]:
Out[44]

LoadError: KeyError: key 44 not found